In [4]:
import pymysql
import requests
import pandas as pd
import numpy as np

### Genera un df con el archivo csv "INPUTEVENTS_MV"

In [6]:
df = pd.read_csv('C:\\Users\\cfari\OneDrive\\Escritorio\\PF_GRUPAL\\DATASET\\INPUTEVENTS_MV.csv', encoding='UTF-8', delimiter=',', decimal='.')
df.head(3)

,row_id,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,amount,amountuom,rate,...,totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_editedby,comments_canceledby,comments_date,originalamount,originalrate
0,118897,42367,139932,250305,2147-10-29 16:45:00,2147-10-29 16:46:00,225799,60.000000,ml,NaN,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,60.0,60.0
1,118898,42367,139932,250305,2147-10-20 13:17:00,2147-10-20 13:18:00,223258,10.000000,units,NaN,...,NaN,0,0,1,Rewritten,NaN,RN,2147-10-20 13:18:00,10.0,10.0
2,118899,42367,139932,250305,2147-10-29 03:23:00,2147-10-29 03:53:00,226089,99.999999,ml,199.999998,...,ml,0,0,0,FinishedRunning,NaN,NaN,NaN,100.0,200.0


In [7]:
df = df.replace(np.nan, 0)

### Se conecta con la BD MySQL 

In [ ]:
con = pymysql.connect(
      host='database-proyecto-final.cpjrl88bhdul.us-east-1.rds.amazonaws.com',
      database='databaseuci',
      user='admin',
      password='datascience'
)

### Inserta los registros del df "INPUTEVENTS_MV" en la BD

In [8]:
sql_insert = ''' INSERT 
                   INTO  INPUTEVENTS_MV
                         (row_id,subject_id,hadm_id,icustay_id,starttime,endtime,itemid,amount,amountuom,rate,rateuom,storetime,cgid,orderid,linkorderid,
                         ordercategoryname,secondaryordercategoryname,ordercomponenttypedescription,ordercategorydescription,patientweight,totalamount,
                         totalamountuom,isopenbag,continueinnextdept,cancelreason,statusdescription,comments_editedby,comments_canceledby,
                         comments_date,originalamount,originalrate)
                 VALUES  (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s) '''
cursor = con.cursor()
for i in range(0, len(df)):
    row = int(df.loc[i, 'row_id'])
    subj = int(df.loc[i, 'subject_id'])
    hadm = int(df.loc[i, 'hadm_id'])
    icus = int(df.loc[i, 'icustay_id'])
    stt = df.loc[i, 'starttime']
    end = df.loc[i, 'endtime']
    itd = int(df.loc[i, 'itemid'])
    amt = float(df.loc[i, 'amount'])
    amm = df.loc[i, 'amountuom']
    rat = float(df.loc[i, 'rate'])
    rtm = df.loc[i, 'rateuom']
    stm = df.loc[i, 'storetime']
    cgi = int(df.loc[i, 'cgid'])
    oid = int(df.loc[i, 'orderid'])
    lkd = int(df.loc[i, 'linkorderid'])
    ocn = df.loc[i, 'ordercategoryname']
    scn = df.loc[i, 'secondaryordercategoryname']
    ocd = df.loc[i, 'ordercomponenttypedescription']
    otd = df.loc[i, 'ordercategorydescription']
    ptt = float(df.loc[i, 'patientweight'])
    tta = float(df.loc[i, 'totalamount'])
    tat = df.loc[i, 'totalamountuom']
    ipg = int(df.loc[i, 'isopenbag'])
    cnd = int(df.loc[i, 'continueinnextdept'])
    crn = int(df.loc[i, 'cancelreason'])
    std = df.loc[i, 'statusdescription']
    cme = df.loc[i, 'comments_editedby']
    cmc = df.loc[i, 'comments_canceledby']
    cmd = df.loc[i, 'comments_date']
    ogm = float(df.loc[i, 'originalamount'])
    oge = float(df.loc[i, 'originalrate'])
    reg_s = (row, subj, hadm, icus, stt, end, itd, amt, amm, rat, rtm, stm, cgi, oid, lkd, ocn, scn, ocd, otd, ptt, tta, tat, ipg, cnd, crn, std, cme, cmc, cmd, ogm, oge)
    cursor.execute(sql_insert, reg_s)
#cursor.fetchone()

### Confirma la inserción de los registros a la BD

In [9]:
con.commit()

### Cierra el cursor y la conexion a la BD

In [10]:
cursor.close()
con.close()